In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )

In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 1min 32s, sys: 1.8 s, total: 1min 34s
Wall time: 1min 37s


In [6]:
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
# "ICD10CM", "MONDO"
condition_1 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_mondo.df['object_id'].str.contains("MONDO")
condition_4 = msdf_mondo.df['subject_id'].str.contains("ICD10CM")
msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified,Precocious puberty
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified,Discordant ventriculoarterial connection
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified,Discordant atrioventricular connection
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified,Disorders of mineral metabolism
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified,Erysipeloid


In [7]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_lex.df['object_id'].str.contains("MONDO")
condition_4 = msdf_lex.df['subject_id'].str.contains("ICD10CM")
msdf_lex.df = msdf_lex.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_lex.df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
128740,ICD10CM:A00,Cholera,skos:exactMatch,MONDO:0015766,cholera,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,cholera
128746,ICD10CM:A01.0,Typhoid fever,skos:exactMatch,MONDO:0005619,typhoid fever,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,typhoid fever
128747,ICD10CM:A01.0,Typhoid fever,skos:exactMatch,MONDO:0005619,typhoid fever,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,typhoid fever
128751,ICD10CM:A01.1,Paratyphoid fever A,skos:closeMatch,MONDO:0018626,paratyphoid fever,semapv:LexicalMatching,oaklib,0.500000,rdfs:label,oio:hasRelatedSynonym,paratyphoid fever a
128755,ICD10CM:A01.2,Paratyphoid fever B,skos:closeMatch,MONDO:0018626,paratyphoid fever,semapv:LexicalMatching,oaklib,0.500000,rdfs:label,oio:hasRelatedSynonym,paratyphoid fever b


In [8]:
duplicate_df_lex = msdf_lex.df.copy()
duplicate_df_mondo = msdf_mondo.df.head().copy()

duplicate_df_lex = duplicate_df_lex.rename(columns={"subject_id": "object_id","object_id": "subject_id" })
duplicate_df_mondo = duplicate_df_mondo.rename(columns={"subject_id": "object_id","object_id": "subject_id" })


msdf_lex.df = pd.concat([duplicate_df_lex, msdf_lex.df]).drop_duplicates()
msdf_mondo.df = pd.concat([duplicate_df_mondo, msdf_mondo.df]).drop_duplicates()
msdf_mondo.df.head()

,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified,Precocious puberty
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified,Discordant ventriculoarterial connection
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified,Discordant atrioventricular connection
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified,Disorders of mineral metabolism
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified,Erysipeloid


In [9]:
comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
comparison_df = comparison_ms_diff.combined_dataframe
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
comparison_df.head()

,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
53210,ICD10CM:C67.9,non-papillary transitional cell carcinoma of t...,skos:broadMatch,MONDO:0016211,Unspecified,"Malignant neoplasm of bladder, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
52437,ICD10CM:D61.9,aplastic anemia,skos:broadMatch,MONDO:0015909,Unspecified,"Aplastic anemia, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
2592,ICD10CM:A75.3,typhus,skos:broadMatch,MONDO:0001246,Unspecified,Typhus fever due to Rickettsia tsutsugamushi,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
8308,ICD10CM:J30,rhinitis,skos:exactMatch,MONDO:0003014,Unspecified,Vasomotor and allergic rhinitis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
52846,ICD10CM:Q36.9,isolated cleft lip,skos:broadMatch,MONDO:0016043,Unspecified,"Cleft lip, unilateral",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [10]:
mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
unmapped_df = comparison_df[
    (comparison_df['comment'].str.contains("|".join(mappings)))
]

unmapped_df.head()

,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
53210,ICD10CM:C67.9,non-papillary transitional cell carcinoma of t...,skos:broadMatch,MONDO:0016211,Unspecified,"Malignant neoplasm of bladder, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
52437,ICD10CM:D61.9,aplastic anemia,skos:broadMatch,MONDO:0015909,Unspecified,"Aplastic anemia, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
2592,ICD10CM:A75.3,typhus,skos:broadMatch,MONDO:0001246,Unspecified,Typhus fever due to Rickettsia tsutsugamushi,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
8308,ICD10CM:J30,rhinitis,skos:exactMatch,MONDO:0003014,Unspecified,Vasomotor and allergic rhinitis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
52846,ICD10CM:Q36.9,isolated cleft lip,skos:broadMatch,MONDO:0016043,Unspecified,"Cleft lip, unilateral",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [11]:
# Let reviewers check if this makes sense or no.
unmapped_lex_exact = unmapped_df[(unmapped_df['comment'] == 'LEXMATCH') & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_lex_exact.to_csv(join(dir_name, "unmapped_lex_exact.tsv"), sep='\t', index = False)
unmapped_lex_exact.head()


,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
134091,ICD10CM:I10,Essential (primary) hypertension,skos:exactMatch,MONDO:0005044,semapv:LexicalMatching,hypertensive disorder,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,high blood pressure
134091,MONDO:0005044,Essential (primary) hypertension,skos:exactMatch,ICD10CM:I10,semapv:LexicalMatching,hypertensive disorder,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,high blood pressure
131772,MONDO:0016239,Cystinosis,skos:exactMatch,ICD10CM:E72.04,semapv:LexicalMatching,cystinosis,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,cystinosis
131773,MONDO:0016239,Cystinosis,skos:exactMatch,ICD10CM:E72.04,semapv:LexicalMatching,cystinosis,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,cystinosis
133760,ICD10CM:H50.61,Brown's sheath syndrome,skos:exactMatch,MONDO:0014624,semapv:LexicalMatching,Brown syndrome,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,brown's sheath syndrome


In [12]:
# Inspect why these are missing from SSSOM mappings

unmapped_mondo_exact = unmapped_df[(unmapped_df['comment'] == 'MONDO_MAPPINGS')& (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))
# Add distances
# Add Levenshtein distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# Add Jaccard Index [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# Add Monge-Elkan Distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
unmapped_mondo_exact.head()


/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_92474/653029316.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))


,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string,levenshtein_dist,jaccard_index,monge_elkan
8308,ICD10CM:J30,rhinitis,skos:exactMatch,MONDO:0003014,Unspecified,Vasomotor and allergic rhinitis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,23,0.741935,1.875000
15539,ICD10CM:F30-F39,mood disorder,skos:exactMatch,MONDO:0005371,Unspecified,Mood [affective] disorders (F30-F39),MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,23,0.638889,1.923077
17598,ICD10CM:J13,streptococcal pneumonia,skos:exactMatch,MONDO:0005972,Unspecified,Pneumonia due to Streptococcus pneumoniae,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,20,0.476190,1.958412
14412,ICD10CM:I10-I16,hypertensive disorder,skos:exactMatch,MONDO:0005044,Unspecified,Hypertensive diseases (I10-I16),MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,14,0.514286,1.954649
14411,ICD10CM:I15,hypertensive disorder,skos:exactMatch,MONDO:0005044,Unspecified,Secondary hypertension,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,20,0.407407,1.954649


In [13]:
combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=msdf_lex.prefix_map, metadata=msdf_lex.metadata)
combined_msdf.df.head()

,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
53210,ICD10CM:C67.9,non-papillary transitional cell carcinoma of t...,skos:broadMatch,MONDO:0016211,Unspecified,"Malignant neoplasm of bladder, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
52437,ICD10CM:D61.9,aplastic anemia,skos:broadMatch,MONDO:0015909,Unspecified,"Aplastic anemia, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
2592,ICD10CM:A75.3,typhus,skos:broadMatch,MONDO:0001246,Unspecified,Typhus fever due to Rickettsia tsutsugamushi,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
8308,ICD10CM:J30,rhinitis,skos:exactMatch,MONDO:0003014,Unspecified,Vasomotor and allergic rhinitis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
52846,ICD10CM:Q36.9,isolated cleft lip,skos:broadMatch,MONDO:0016043,Unspecified,"Cleft lip, unilateral",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [ ]:
%%time
df_dict = split_dataframe(combined_msdf)

In [ ]:
mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
mondo_icd_list

In [ ]:
for match in mondo_icd_list:
    fn = match + ".tsv"
    df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [ ]:
df_dict['mondo_exactmatch_icd10cm'].df